## Housing price

https://storage.googleapis.com/ml_universities/california_housing_train.csv

что хочу:
    - импортировать в пандас, посмотреть что как, очистить, разбить на трейн и тест
    - построить TF датасет, поработать с фичами
    - построить и обучить денс-нн, учить с коллбеками, с сохранением слоев и с валидацией
    - на коллбеках сохраняться

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt

%matplotlib inline
%load_ext tensorboard

In [2]:
df = pd.read_csv("https://storage.googleapis.com/ml_universities/california_housing_train.csv", sep=",")

In [3]:
df.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [4]:
# разбиваем на трейн и валидацию
df_train, df_test = train_test_split(df, train_size=0.8)

In [5]:
df_train.describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000,13600.000000
mean,-119.569421,35.629117,28.574118,2640.470221,538.320000,1429.469044,500.272206,3.887688,207351.995882
std,2.009109,2.140706,12.556721,2179.448642,419.338846,1144.163835,382.340709,1.903478,115891.866754
min,-124.350000,32.540000,1.000000,2.000000,2.000000,3.000000,2.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1461.000000,295.000000,787.000000,281.000000,2.569300,119475.000000
50%,-118.490000,34.250000,29.000000,2127.000000,433.000000,1164.000000,408.000000,3.547100,180800.000000
75%,-118.000000,37.710000,37.000000,3153.000000,648.000000,1722.000000,606.000000,4.782350,265300.000000
max,-114.310000,41.880000,52.000000,37937.000000,5471.000000,35682.000000,5189.000000,15.000100,500001.000000


In [6]:
housing_median_age_mean = df_train['housing_median_age'].mean()
housing_median_age_std =  df_train['housing_median_age'].std()

median_income_mean = df_train['median_income'].mean()
median_income_std = df_train['median_income'].std()

# эти данные буду использовать для нормализации
housing_median_age_mean, housing_median_age_std, median_income_mean, median_income_std

(28.574117647058824,
 12.556721192543456,
 3.8876876029411735,
 1.9034783629772813)

In [7]:
# то, что предсказываем
target_train = df_train.pop('median_house_value')
target_test = df_test.pop('median_house_value')

## Работа с фичами
    
    - хочу на лету создавать новые фичи - среднее число комнат на дом, среднее число жильцов на комнату
    - longitude и latitude сначала делаем бакетами, потом делаем из них крест. Т.к у нас признаком являются не сами числа, а принадлежность к квадратику
    - нормализовать все данными из датасета

In [8]:
features = ['longitude', 'latitude', 'housing_median_age', 'population'
            'total_rooms', 'total_bedrooms','households','median_income']

In [9]:
# прекрасно, прекрасно справляется :)
def make_new_features(x, y):
    x['family_size'] = x['population'] / x['households']
    x['pop_per_room'] = x['population'] / x['total_rooms']
    x['pop_per_bedroom'] = x['population'] / x['total_bedrooms']
    return x, y

In [10]:
# нам это нужно и для трейна и для предикта
def create_dataset(d, t):
    ds = tf.data.Dataset.from_tensor_slices((d.to_dict('list'), t.values)).batch(64).shuffle(100).repeat()
    ds_featured = ds.map(make_new_features)
    return ds_featured
    #layer = tf.keras.layers.DenseFeatures(features_list)
    
ds = create_dataset(df_train, target_train)
ds_train = create_dataset(df_train, target_train)
ds_test = create_dataset(df_test, target_test)

In [11]:
for x, y in ds.take(1):
    print(x, y)

{'longitude': <tf.Tensor: shape=(64,), dtype=float32, numpy=
array([-121.89, -118.11, -118.12, -121.45, -117.04, -121.47, -124.13,
       -124.07, -118.18, -121.12, -121.84, -119.72, -118.3 , -121.3 ,
       -116.93, -122.18, -121.94, -118.29, -121.99, -117.01, -118.3 ,
       -118.43, -115.54, -117.2 , -118.13, -121.24, -122.33, -118.29,
       -122.23, -117.3 , -121.4 , -118.47, -122.46, -116.83, -118.26,
       -118.3 , -117.85, -118.97, -122.25, -117.95, -119.29, -118.13,
       -121.78, -122.18, -119.08, -118.44, -122.04, -118.34, -122.91,
       -116.9 , -121.04, -122.42, -122.09, -118.49, -116.39, -119.08,
       -120.45, -122.15, -119.03, -118.73, -117.93, -122.15, -122.26,
       -117.09], dtype=float32)>, 'latitude': <tf.Tensor: shape=(64,), dtype=float32, numpy=
array([37.36, 34.02, 34.14, 38.63, 32.54, 38.53, 40.55, 40.87, 33.78,
       38.  , 37.32, 34.42, 33.78, 38.65, 32.87, 37.72, 39.45, 34.11,
       36.96, 32.99, 33.99, 34.3 , 32.79, 34.24, 33.85, 38.01, 37.56,
      

In [12]:
# feature_column имеют немного другую роль. Они потом превращаются в слой модели,
# т.е по сути на старте они ничего и не знают про датасет, модель будет пихать в них
# данные как в мясорубку

num_features_unnorm = ['family_size', 'pop_per_room', 'pop_per_bedroom',
                       'total_rooms', 'total_bedrooms', 'households']

def create_features():
    # не важно что мы тут делаем, важно что выводим в виде итогового массива
    latitude = tf.feature_column.bucketized_column(
            tf.feature_column.numeric_column('latitude'), boundaries = np.arange(32.0, 42, 1).tolist())    
    
    longitude = tf.feature_column.bucketized_column(
            tf.feature_column.numeric_column('longitude'), boundaries = np.arange(-122, -114, 1).tolist())  
    
    squares = tf.feature_column.indicator_column(
        tf.feature_column.crossed_column([latitude, longitude], hash_bucket_size=1000))

    # нормализатор там встроен, пробую
    def norm_housing_median_age(x):
        return (x - housing_median_age_mean)/ housing_median_age_std
    housing_median_age = tf.feature_column.numeric_column('housing_median_age', 
                                                          normalizer_fn = norm_housing_median_age)

    def norm_median_income(x):
        return (x - median_income_mean)/ median_income_std
    median_income = tf.feature_column.numeric_column('median_income', 
                                                          normalizer_fn = norm_median_income)

    num_features = [tf.feature_column.numeric_column(name) for name in num_features_unnorm]
    
    return [housing_median_age, median_income, squares] + num_features

In [13]:
# DenseFeatures по сути просто делает тензор, на входе ему нужно подать массив "ленточек" из которых делать
# а ленточки определяются через feature_columns

layer = tf.keras.layers.DenseFeatures(create_features())

In [14]:
for x, y in ds.take(1):
    print(layer(x).numpy())

[[ 6.3121214e+00  3.3000000e+02  4.3210983e-01 ...  1.1841955e+00
   3.5300000e+02  1.7590000e+03]
 [ 2.8150470e+00  9.5700000e+02  3.5247120e-01 ...  4.1452530e-01
   9.9800000e+02  6.4990000e+03]
 [ 3.1012270e+00  6.5200000e+02 -1.2403014e+00 ...  5.5918139e-01
   6.7200000e+02  3.6160000e+03]
 ...
 [ 1.9844790e+00  4.5100000e+02 -1.3995786e+00 ...  4.9611974e-01
   5.0600000e+02  1.8040000e+03]
 [ 2.9375000e+00  9.6000000e+01 -1.4792172e+00 ...  5.4651165e-01
   1.0700000e+02  5.1600000e+02]
 [ 2.1310241e+00  6.6400000e+02  1.8656051e+00 ...  5.7755101e-01
   7.4100000e+02  2.4500000e+03]]


## модель, обучение, коллбеки и вот это вот все

In [15]:
model = tf.keras.Sequential([
    layer,
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(512, activation='relu'), 
    tf.keras.layers.Dense(1)
])

optimizer = tf.optimizers.Adam()

model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=['mse', 'mae'])

In [16]:
# сохранение
save_callback = tf.keras.callbacks.ModelCheckpoint('./save/model.ckpt', save_weights_only=True, verbose=1)

# тензорбоард
tensorboard = tf.keras.callbacks.TensorBoard(
    log_dir='./save/', histogram_freq=0, write_graph=True, write_images=False)


In [17]:
history = model.fit(ds_train, epochs=5, steps_per_epoch=212, 
                    validation_data=ds_test, validation_steps=45, callbacks=[save_callback, tensorboard])

Epoch 1/5
212/212 [==============================] - ETA: 0s - loss: 32624465920.0000 - mse: 32624465920.0000 - mae: 136587.7031
Epoch 00001: saving model to ./save/model.ckpt
212/212 [==============================] - 22s 102ms/step - loss: 32624465920.0000 - mse: 32624465920.0000 - mae: 136587.7031 - val_loss: 28131969024.0000 - val_mse: 28131969024.0000 - val_mae: 122342.4922
Epoch 2/5
212/212 [==============================] - ETA: 0s - loss: 27353038848.0000 - mse: 27353038848.0000 - mae: 120440.7734
Epoch 00002: saving model to ./save/model.ckpt
212/212 [==============================] - 17s 78ms/step - loss: 27353038848.0000 - mse: 27353038848.0000 - mae: 120440.7734 - val_loss: 26471413760.0000 - val_mse: 26471413760.0000 - val_mae: 120026.1328
Epoch 3/5
212/212 [==============================] - ETA: 0s - loss: 27258079232.0000 - mse: 27258079232.0000 - mae: 120262.7422
Epoch 00003: saving model to ./save/model.ckpt
212/212 [==============================] - 18s 87ms/step - lo

In [ ]:
!tensorboard --logdir='./save'


In [18]:
model.predict(ds_test)

ValueError: When providing an infinite dataset, you must specify the number of steps to run.

In [ ]:
model.comp